In [1]:
!wget -q https://archive.ics.uci.edu/static/public/352/online+retail.zip
!unzip online+retail.zip
!pip install -q -U openpyxl

Archive:  online+retail.zip
 extracting: Online Retail.xlsx      


In [3]:
import pandas as pd

In [4]:
df = pd.read_excel("Online Retail.xlsx", dtype={'InvoiceNo': str, 'StockCode': str, 'Description': str})

In [5]:
df_descr_null = df[df['Description'].isnull()]
df_descr_null

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,84581,NaN,-2,2011-12-07 18:26:00,0.0,NaN,United Kingdom
535326,581203,23406,NaN,15,2011-12-07 18:31:00,0.0,NaN,United Kingdom
535332,581209,21620,NaN,6,2011-12-07 18:35:00,0.0,NaN,United Kingdom
536981,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom


In [6]:
%%time
for index, row in df_descr_null.iterrows():
    stock_code = row['StockCode']
    # Find the most frequent description for this StockCode
    most_frequent_description = df[df['StockCode'] == stock_code]['Description'].mode()
    if not most_frequent_description.empty:
        df.at[index, 'Description'] = most_frequent_description[0]   

CPU times: user 37.8 s, sys: 51.6 ms, total: 37.9 s
Wall time: 37.9 s


In [7]:
%%time
most_freq = (
    df[["StockCode", "Description"]]
      .value_counts()
      .reset_index()
      .groupby("StockCode")
      .head(1)
)
most_freq.columns = ["StockCode", "freq_Description", "frequency"]

df = df.merge(most_freq, on="StockCode", how="left")
df["Description"] = (
    df["Description"].mask(
       df["Description"].isnull(),
       df["freq_Description"]
    )
)
df.drop(columns = ["freq_Description", "frequency"], inplace=True)

CPU times: user 246 ms, sys: 12 ms, total: 258 ms
Wall time: 255 ms


In [8]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
